In [13]:
"""
robust_dem_fill.py

Fills NoData pixels in a DEM (continuous raster) using nearest-valid-pixel filling
implemented with scipy.ndimage.distance_transform_edt.

Usage:
    python robust_dem_fill.py /path/to/dem_aligned.tif /path/to/dem_filled.tif

Notes:
- Assumes single-band DEM.
- Detects nodata from metadata (src.nodata). If metadata nodata is None,
  it will treat NaNs as missing. If neither exists, it will exit (no-op).
- Output raster will be float32 and have no nodata (filled).
"""
import sys
from pathlib import Path

import numpy as np
import rasterio
from scipy import ndimage


def fill_dem_nodata_nearest(input_tif: str | Path, output_tif: str | Path) -> None:
    input_tif = str(input_tif)
    output_tif = str(output_tif)

    with rasterio.open(input_tif) as src:
        # Read first band as float64 (higher precision during processing)
        arr = src.read(1).astype(np.float64)
        meta = src.meta.copy()
        nodata = src.nodata

        print(f"Input: {input_tif}")
        print(f"Raster shape (h x w): {arr.shape}")
        print(f"Metadata nodata: {nodata}")

    # Build boolean mask of missing pixels (True == missing)
    if nodata is not None:
        missing = (arr == nodata)
    else:
        # If nodata not defined, treat NaNs as missing.
        missing = np.isnan(arr)

    missing_count = int(np.sum(missing))
    total = arr.size
    pct = 100.0 * missing_count / total if total else 0.0
    print(f"Total pixels: {total:,}")
    print(f"Missing pixels: {missing_count:,} ({pct:.2f}%)")

    if missing_count == 0:
        print("No missing pixels detected — copying input to output.")
        # simple copy
        with rasterio.open(input_tif) as src:
            profile = src.profile
            profile.update(dtype=rasterio.float32)
            with rasterio.open(output_tif, "w", **profile) as dst:
                dst.write(src.read(1).astype(np.float32), 1)
        print(f"Saved (unchanged) to: {output_tif}")
        return

    # Make sure mask is boolean ndarray
    missing = missing.astype(bool)

    # distance_transform_edt expects an array where zeros indicate "objects"
    # We want the nearest valid pixel for each missing pixel.
    # If missing==True at missing positions, then zeros are valid positions (where missing==False).
    # So pass missing (True on missing) to distance_transform_edt: it computes distance to nearest zero (valid).
    try:
        # returns: distances (2D), and indices (ndim x H x W)
        dist, inds = ndimage.distance_transform_edt(missing, return_distances=True, return_indices=True)
    except Exception as e:
        raise RuntimeError("distance_transform_edt failed — ensure scipy is installed and enough memory is available.") from e

    # inds shape is (ndim, H, W) where ndim==2 for 2D arrays.
    # inds[0] -> nearest-row indices (y), inds[1] -> nearest-col indices (x)
    iy = inds[0].astype(np.int64)
    ix = inds[1].astype(np.int64)

    # nearest_values[y,x] gives, for every pixel, the value of the nearest valid pixel.
    # For missing pixels we will assign nearest_values[missing] -> filled[missing]
    nearest_values = arr[iy, ix]

    # Copy input and fill missing positions
    filled = arr.copy()
    filled[missing] = nearest_values[missing]

    # Write output: set dtype float32 and remove nodata (since we've filled)
    out_meta = meta.copy()
    out_meta.update(dtype=rasterio.float32, nodata=None, count=1)

    with rasterio.open(output_tif, "w", **out_meta) as dst:
        dst.write(filled.astype(np.float32), 1)

    print(f"✅ Filled missing pixels and saved to: {output_tif}")


if __name__ == "__main__":
    if len(sys.argv) < 3:
        
        sys.exit(1)
    in_tif = "C:\\Users\\Ankit\\OneDrive\\Desktop\\merged_DEM_30m_32644_aligned.tif"
    out_tif = "C:\\Users\\Ankit\\OneDrive\\Desktop\\merged_DEM_30m_32644_aligned_filled.tif"

    fill_dem_nodata_nearest(in_tif, out_tif)


Input: C:\Users\Ankit\OneDrive\Desktop\merged_DEM_30m_32644_aligned.tif
Raster shape (h x w): (11904, 13604)
Metadata nodata: 255.0
Total pixels: 161,942,016
Missing pixels: 8,838,151 (5.46%)
✅ Filled missing pixels and saved to: C:\Users\Ankit\OneDrive\Desktop\merged_DEM_30m_32644_aligned_filled.tif
